# Agent Wrapper Demo - LLM Provider Abstraction

This notebook demonstrates how to use Docs2Synth's agent wrapper to seamlessly switch between different LLM providers.


## 1. Import Required Modules


In [ ]:
import os

os.environ["DOCS2SYNTH_CONFIG"] = "../../config.yml"

In [ ]:
from docs2synth.agent import AgentWrapper, QAGenerator

## 2. View Available Providers


In [ ]:
from docs2synth.agent.providers import PROVIDER_REGISTRY

print("Available Providers:")
for name, provider_class in sorted(PROVIDER_REGISTRY.items()):
    print(f"  - {name}: {provider_class.__name__}")


## 3. Basic Usage - Text Generation


In [ ]:
agent = AgentWrapper(provider="openai", model="gpt-3.5-turbo")
response = agent.generate("Explain what artificial intelligence is in one sentence")
print(f"Response: {response.content}")
print(f"Token Usage: {response.usage}")


## 4. QA Generation - Main Feature


In [ ]:
# Example document content
content = """
Python is a high-level programming language first released by Guido van Rossum in 1991.
It is known for its simple and readable syntax, making it ideal for beginners.
Python supports multiple programming paradigms including object-oriented, imperative,
functional, and procedural programming. It has a large standard library and a rich
ecosystem of third-party packages.
"""

print("Example Content:")
print(content.strip())


In [ ]:
# Generate QA pair (automatically uses JSON format)
# Uncomment and set your API key to run

generator = QAGenerator(provider="openai", model="gpt-3.5-turbo")
qa_pair = generator.generate_qa_pair(content)

print(f"\nQuestion: {qa_pair['question']}")
print(f"Answer: {qa_pair['answer']}")


## 5. Batch QA Generation


In [ ]:
# Multiple document contents
contents = [
    "Machine learning is a subset of artificial intelligence focused on enabling computers to learn from data.",
    "Deep learning uses multi-layer neural networks to simulate how the human brain works.",
    "Natural language processing enables computers to understand and generate human language."
]

# Batch generation
generator = QAGenerator(provider="openai", model="gpt-3.5-turbo")
qa_pairs = generator.generate_qa_pairs(contents)

for i, qa in enumerate(qa_pairs, 1):
    print(f"\n=== QA Pair {i} ===")
    print(f"Question: {qa['question']}")
    print(f"Answer: {qa['answer']}")


## 6. Switching Between Different Providers


In [ ]:
# All providers use the same interface - just change the provider parameter to switch

# OpenAI
generator = QAGenerator(provider="openai")

# Anthropic Claude
generator = QAGenerator(provider="anthropic")

# Google Gemini
generator = QAGenerator(provider="gemini")

# Doubao (Bean)
generator = QAGenerator(
    provider="doubao"
)

# Local Ollama (requires local ollama serve running)
generator = QAGenerator(provider="ollama")

# Local Hugging Face
generator = QAGenerator(
    provider="huggingface"
)


## 7. Using JSON Format Output


In [ ]:
# Use AgentWrapper directly and specify JSON format
agent = AgentWrapper(provider="openai", model="gpt-3.5-turbo")

response = agent.generate(
    "Return a JSON object with 'name' and 'age' fields, name='John', age=25",
    response_format="json"
)

print("JSON Response:", response.content)
data = json.loads(response.content)
print("Parsed Data:", data)



## 8. Chat Interface Example


In [ ]:
agent = AgentWrapper(provider="openai", model="gpt-3.5-turbo")

messages = [
    {"role": "user", "content": "What is Python?"},
    {"role": "assistant", "content": "Python is a programming language."},
    {"role": "user", "content": "What are its characteristics?"}
]

response = agent.chat(messages)
print("Response:", response.content)


## 9. Runtime Provider Switching


In [ ]:
agent = AgentWrapper(provider="openai", model="gpt-3.5-turbo")
print(f"Current Provider: {agent.provider_type}, Model: {agent.model}")

# Switch to Ollama
agent.switch_provider("ollama", model="llama2")
print(f"Switched Provider: {agent.provider_type}, Model: {agent.model}")


## 10. QA Generation with Verification


In [ ]:
generator = QAGenerator(provider="openai", model="gpt-3.5-turbo")

# Generate and verify (meaningfulness + correctness)
qa_pair = generator.generate_with_verification(
    content,
    meaningful_check=True,
    correctness_check=True
)

if qa_pair:
    print("✓ QA pair passed verification")
    print(f"Question: {qa_pair['question']}")
    print(f"Answer: {qa_pair['answer']}")
else:
    print("✗ QA pair failed verification")


## Notes

1. **API Keys**: Make sure to set the correct API keys in `config.yml` or use environment variables
2. **Local Models**: Ollama and Hugging Face require local setup or model installation
3. **JSON Mode**: QAGenerator automatically uses JSON mode to ensure responses can be parsed correctly
4. **Provider Differences**: Different providers implement JSON mode differently (native support vs prompt engineering)
